In [1]:
from pynq.overlays.hdmiin import HDMIINOverlay
overlay = HDMIINOverlay("hdmiin.bit")

load ready


In [2]:
from pprint import pprint
pprint(overlay.ip_dict)

{'btns_gpio': {'addr_range': 65536,
               'driver': <class 'pynq.lib.axigpio.AxiGPIO'>,
               'fullpath': 'btns_gpio',
               'gpio': {},
               'interrupts': {'ip2intc_irpt': {'controller': 'system_interrupts',
                                               'fullpath': 'btns_gpio/ip2intc_irpt',
                                               'index': 10}},
               'phys_addr': 1092616192,
               'state': None,
               'type': 'xilinx.com:ip:axi_gpio:2.0'},
 'h264enc_with_axi_0': {'addr_range': 65536,
                        'driver': <class 'pynq.overlay.DefaultIP'>,
                        'fullpath': 'h264enc_with_axi_0',
                        'gpio': {},
                        'interrupts': {},
                        'phys_addr': 2210398208,
                        'state': None,
                        'type': 'xilinx.com:user:h264enc_with_axi:1.0'},
 'rgbleds_gpio': {'addr_range': 65536,
                  'driver': <class

In [3]:
print(hex(overlay.ip_dict['h264enc_with_axi_0']['phys_addr']))

0x83c00000


In [4]:
from pynq import Xlnk

xlnk = Xlnk()

In [5]:
import numpy as np
data_size = int(640*480*1.5)
cma_send = xlnk.cma_array([1, data_size], dtype=np.uint8)

In [6]:
with open('input_640_480.rgb32', 'rb') as f:
    image_data = f.read()

In [7]:
for i in range(data_size):
    cma_send[0][i] = image_data[i]

In [8]:
for i in range(10):
    print(hex(cma_send[0][i]))

0x66
0x87
0xb7
0xff
0x67
0x87
0xb6
0xff
0x66
0x87


In [9]:
hex(cma_send.physical_address)

'0x16880000'

In [10]:
cma_recv = xlnk.cma_array([1, data_size], dtype=np.uint8)

In [11]:
hex(cma_recv.physical_address)

'0x16900000'

In [12]:
from pynq import MMIO

h264 = MMIO(overlay.ip_dict['h264enc_with_axi_0']['phys_addr'], 0x10000)

In [13]:
h264.write(0x04, 0x00001d27)
h264.write(0x08, 0)
h264.write(0x40, 0x00008816)
h264.write(0x14, cma_send.physical_address)
h264.write(0x18, cma_recv.physical_address)
h264.write(0x00, 1)

In [14]:
for i in range(10):
    print(hex(cma_recv[0][i]))

0xfb
0x84
0x78
0x43
0xfc
0x0
0x54
0x67
0xb
0x1b


In [15]:
print(h264.read(0x1c))

217414


In [16]:
hex(1136787456)

'0x43c20000'